In [1]:
import sys
sys.path.append('../')
import os
import cv2
import numpy as np
import glob
import matplotlib.pyplot as plt
from libraries.Matcher import Matcher
from libraries.Detector import Detector
from libraries.FileManagerKPDS import FileManagerKPDS as fm
from libraries.listdirNoHidden import listdirNoHidden as ldnh
import time

In [2]:
datasetPath = '../../datasets/rescaled_dataset/'
newDatasetPath = '../../datasets/cropped_dataset/'
method = 'ORB'
detector = Detector.defineDetector(method)


In [3]:
datasets = ["combined_dataset", "zoomed_dataset", "downscaled_dataset", "gBlur_dataset", "gNoise_dataset", "hor_dataset", "ver_dataset"]
cached_dataset = {}
pathList = ldnh(os.path.join(datasetPath, 'images'))

for imgPath in pathList:
	imgName = os.path.basename(imgPath)
	img1 = cv2.imread(imgPath)
	kp1 = fm.loadKP(datasetPath, imgName, method)
	des1 = fm.loadDS(datasetPath, imgName, method)
	cached_dataset[imgName] = (img1, kp1, des1)


In [4]:
# for i, dictItem in enumerate(cached_dataset.items()):
#   imgName = dictItem[0]
#   img1, kp1, des1 = dictItem[1]

#   #show image
#   print(imgName)
#   plt.figure(figsize=(20,10))
#   plt.imshow(img1)
#   plt.show()



In [5]:

with open('all'+method+'1.txt', 'w') as file: 
	startTime = time.time()
	totImages = 0
	for dataset in datasets:
		newDatasetPath = '../../datasets/' + dataset + '/'
		newDatasetImagesPaths = ldnh(newDatasetPath)
		totImages += len(newDatasetImagesPaths)
		mapArray = []
		mapArrayNormalized = []
		for j, newImgPath in enumerate(newDatasetImagesPaths):
			img2 = cv2.imread(newImgPath)
			newImgName = newImgPath.split('/')[-1] #sempre precachabile, ma è "solo" O(n^2) (stringhe in python sono lente)
			kp2, des2 = detector.detectAndCompute(img2, None)
			scores = [None] * len(cached_dataset)
			normalizedScores = [None] * len(cached_dataset)
			for i, dictItem in enumerate(cached_dataset.items()):
				imgName = dictItem[0]
				img1, kp1, des1 = dictItem[1]
				try: 
					matches = Matcher.getMatches(des1, des2,method) 
					good, mask = Matcher.lowe(matches,0.7)
					scores[i] = (imgName, len(good),  img1, kp1, img2, kp2, matches, mask)
					normalizedScores[i] = (imgName, len(good)/len(matches),  img1, kp1, img2, kp2, matches, mask)
				except:
					scores[i] = (imgName, 0,  img1, kp1, img2, kp2, None, None)
					normalizedScores[i] = (imgName, 0,  img1, kp1, img2, kp2, None, None)

			scores.sort(key=lambda tup: tup[1], reverse=True)
			normalizedScores.sort(key=lambda tup: tup[1], reverse=True)
			#Matcher.showMatches(scores[0][2], scores[0][3], scores[0][4], scores[0][5], scores[0][6], scores[0][7])
			#Matcher.showMatches(normalizedScores[0][2], normalizedScores[0][3], normalizedScores[0][4], normalizedScores[0][5], normalizedScores[0][6], normalizedScores[0][7])

			f0 = 0
			f1 = 0
			combinedTransforms = ""
			if(dataset == 'combined_dataset'):
				splits = newImgName.split('_', 1)
				combinedTransforms = splits[0]+"_"
				newImgName = splits[1]
			while(not (f1 and f0)):
				for i in range(len(cached_dataset)):
					s = 1/(i+1)
					if(scores[i][0] == newImgName):
						mapArray.append(s)
						f0 = 1
					if(normalizedScores[i][0] == newImgName):
						mapArrayNormalized.append(s)
						f1 = 1
			file.write(combinedTransforms+newImgName + " " + str(mapArray[-1])+ " " + str(mapArrayNormalized[-1] )+ "\n") # str(obj) -> obj.__str__()
			print(combinedTransforms+newImgName + " " +str(mapArray[-1])+" "+ str(mapArrayNormalized[-1]))
		file.write("MAP: "+ str(np.mean(mapArray))+ " Dataset: "+ dataset+"\n")
		print("MAP: "+ str(np.mean(mapArray))+ " Dataset: "+ dataset+"\n")
		file.write("MAP Normalized: "+ str(np.mean(mapArrayNormalized))+ " Dataset: "+ dataset+"\n")
		print("MAP Normalized: "+ str(np.mean(mapArrayNormalized))+ " Dataset: "+ dataset+"\n")

	stopTime = time.time()

	file.write("Time: "+ str(stopTime - startTime))
	file.write("Time per query: "+ str((stopTime - startTime)/totImages))




    



7GnZoVeGb_20221206_091309.jpg 0.25 0.2
3VeDo_photo_2022-10-13_16-17-12.jpg 0.1 0.07142857142857142
8GbGn_photo_2022-10-13_16-17-12.jpg 1.0 1.0
3GnHoGb_photo_2022-10-13_16-16-46.jpg 0.2 0.2
1DoZoHo_20221103_080342.jpg 0.03333333333333333 0.058823529411764705
0HoGn_20221206_091312.jpg 0.5 1.0
3GnZoVeGb_20221205_122031.jpg 0.07142857142857142 0.08333333333333333
